In [14]:
import pandas as pd

# 讀取CSV檔案
csv_file_path = 'facebook/20230404_20240403.csv' #facebook平台
file=open(csv_file_path,"rb")  
# 直接使用文件路径读取CSV文件
df = pd.read_csv(csv_file_path,encoding='latin-1',dtype=str)


print("預計爬取清單\n",df['URL'])


預計爬取清單
 0      https://www.facebook.com/160630887998827/posts...
1      https://www.facebook.com/114517875225866/posts...
2      https://www.facebook.com/88683275177/posts/101...
3      https://www.facebook.com/100044425449818/posts...
4      https://www.facebook.com/60182072999/posts/101...
                             ...                        
619    https://www.facebook.com/632576423454215/posts...
620    https://www.facebook.com/groups/10751114314476...
621    https://www.facebook.com/groups/11193809947634...
622    https://www.facebook.com/groups/30378797308649...
623    https://www.facebook.com/groups/60499648623717...
Name: URL, Length: 624, dtype: object


In [15]:
import os

# 設定你想要檢查和創建的資料夾路徑
folder_path = csv_file_path.replace('.csv','/')

# 檢查資料夾是否存在
if not os.path.exists(folder_path):
    # 如果資料夾不存在，則創建它
    os.makedirs(folder_path)
    print(f"資料夾 '{folder_path}' 已創建。")
else:
    # 如果資料夾已存在，則輸出提示信息
    print(f"資料夾 '{folder_path}' 已存在。")

資料夾 'facebook/2024-05-14-16-22-19-CST-search-csv-export/' 已創建。


In [10]:
##存一張圖片存該圖片評論檔成一個csv 以圖片id命名

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time

comtext = []
error = []
imgid = 0

errordf = {
    'id': [],
    'error': []
}

for i in df['URL']:
    
    try:
        driver = webdriver.Chrome()
        # 請替換成你要訪問的網頁的URL
        driver.get(i)
        #等待網站載入 等6秒
        driver.implicitly_wait(6)
        #點叉叉關掉登入框
        driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]').click()

        ##分離id
        imgid = i.split('posts/')[1]
        print('圖片id',imgid)
        
        ##分離id
        
        
        ##取圖片
        t = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/div[3]/div[2]/div[1]/a/div[1]/div/div/div/img')
        t1 = t.get_attribute("outerHTML")
        #print(t1)

        # 使用 BeautifulSoup 解析 HTML 并获取 src 属性
        soup = BeautifulSoup(t1, 'html.parser')
        t2 = soup.img['src']  # 提取图片的 URL
        print(t2)

        pic = requests.get(t2)
        print(pic)
        img2 = pic.content #圖片裡的內容
        imgpath = csv_file_path.replace('.csv','/')
        imgpath = imgpath + str(imgid) + '.jpg'
        print(imgpath)
        pic_out = open(imgpath,'wb') 
        pic_out.write(img2) #將get圖片存入img2
        pic_out.close() #關閉檔案(很重要)
        ##取圖片
        
        
        ##取評論
        child_elements_comment = 0
        child_elements_name = 0

        #name_com = []

        id_comment = {
            'Name': [],
            'Comment': []
            }

        # 使用CSS选择器定位父元素，返回一个元素列表
        parent_elements = driver.find_elements(By.CSS_SELECTOR, '.x1y1aw1k.xn6708d.xwib8y2.x1ye3gou')

        # 遍历所有父元素
        for parent in parent_elements:
            # 在每个父元素下查找评论子元素
            child_elements_comment = parent.find_elements(By.CLASS_NAME, 'x1lliihq.xjkvuk6.x1iorvi4')
            # 在每个父元素下查找名字子元素
            child_elements_name = parent.find_elements(By.CLASS_NAME, 'xt0psk2')

            # 从找到的子元素中提取文本，这里假设每个父元素只有一个名字和一个评论
            for comment, name in zip(child_elements_comment, child_elements_name):
                
                id_comment['Name'].append(name.text)
                id_comment['Comment'].append(comment.text)

                print("Name:", name.text)
                print("Comment:", comment.text)
                print('-------')
                #name_com.append(pair) #原存評論

        #個別存評論    
        # 将数据转换成DataFrame
        dfok = pd.DataFrame(id_comment)
        print(dfok)

        # 将DataFrame保存为CSV文件
        idimgcsv = folder_path + imgid + '.csv' 
        dfok.to_csv(idimgcsv, index=False,encoding='utf-8')
        #個別存評論 
        
        driver.close()




        
        
        
    except Exception as e:
        errordf['id'].append(imgid)
        errordf['error'].append(f"发生错误：{e}")

        #error.append(imgid) 
        print('失敗',imgid,i)
        print(f"发生错误：{e}")
    time.sleep(6)

from pandas.core.frame import DataFrame
# 将数据转换成DataFrame
dfer = pd.DataFrame(errordf)
print(len(errordf),dfer)

#存起來 失敗
nameerror = csv_file_path.replace('.csv','/') + 'error.csv'
# 将DataFrame保存为CSV文件
dfer.to_csv(nameerror, index=False,encoding='utf-8')